In [1]:
!pip install gym==0.23.1
!pip install pygame

import os
import sys
from typing import Optional, Union, List, Tuple
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import random
from tqdm import tqdm
import einops
from pathlib import Path
import matplotlib.pyplot as plt
import gym
import gym.envs.registration
import gym.spaces

Arr = np.ndarray
max_episode_steps = 1000
N_RUNS = 200

# Make sure exercises are in the path
chapter = r"chapter2_rl"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part1_intro_to_rl"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

import part1_intro_to_rl.utils as utils
import part1_intro_to_rl.tests as tests
from plotly_utils import imshow

MAIN = __name__ == "__main__"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
ObsType = int
ActType = int

class MultiArmedBandit(gym.Env):
    action_space: gym.spaces.Discrete
    observation_space: gym.spaces.Discrete
    num_arms: int
    stationary: bool
    arm_reward_means: np.ndarray
    best_arm: int

    def __init__(self, num_arms=10, stationary=True):
        super().__init__()
        self.num_arms = num_arms
        self.stationary = stationary
        self.observation_space = gym.spaces.Discrete(1)
        self.action_space = gym.spaces.Discrete(num_arms)
        self.reset()

    def step(self, arm: ActType) -> Tuple[ObsType, float, bool, dict]:
        '''
        Note: some documentation references a new style which has (termination, truncation) bools in place of the done bool.
        '''
        assert self.action_space.contains(arm)
        if not self.stationary:
            q_drift = self.np_random.normal(loc=0.0, scale=0.01, size=self.num_arms)
            self.arm_reward_means += q_drift
            self.best_arm = int(np.argmax(self.arm_reward_means))
        reward = self.np_random.normal(loc=self.arm_reward_means[arm], scale=1.0)
        obs = 0
        done = False
        info = dict(best_arm=self.best_arm)
        return (obs, reward, done, info)

    def reset(
        self, seed: Optional[int] = None, return_info=False, options=None
    ) -> Union[ObsType, Tuple[ObsType, dict]]:
        super().reset(seed=seed)
        if self.stationary:
            self.arm_reward_means = self.np_random.normal(loc=0.0, scale=1.0, size=self.num_arms)
        else:
            self.arm_reward_means = np.zeros(shape=[self.num_arms])
        self.best_arm = int(np.argmax(self.arm_reward_means))
        if return_info:
            return (0, dict())
        else:
            return 0

    def render(self, mode="human"):
        assert mode == "human", f"Mode {mode} not supported!"
        bandit_samples = []
        for arm in range(self.action_space.n):
            bandit_samples += [np.random.normal(loc=self.arm_reward_means[arm], scale=1.0, size=1000)]
        plt.violinplot(bandit_samples, showmeans=True)
        plt.xlabel("Bandit Arm")
        plt.ylabel("Reward Distribution")
        plt.show()

In [3]:
gym.envs.registration.register(
    id="ArmedBanditTestbed-v0",
    entry_point=MultiArmedBandit,
    max_episode_steps=max_episode_steps,
    nondeterministic=True,
    reward_threshold=1.0,
    kwargs={"num_arms": 10, "stationary": True},
)

env = gym.make("ArmedBanditTestbed-v0")
print(f"Our env inside its wrappers looks like: {env}")

Our env inside its wrappers looks like: <TimeLimit<OrderEnforcing<MultiArmedBandit<ArmedBanditTestbed-v0>>>>


/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment ArmedBanditTestbed-v0
  logger.warn(f"Overriding environment {id}")


In [4]:
class Agent:
    '''
    Base class for agents in a multi-armed bandit environment

    (you do not need to add any implementation here)
    '''
    rng: np.random.Generator

    def __init__(self, num_arms: int, seed: int):
        self.num_arms = num_arms
        self.reset(seed)

    def get_action(self) -> ActType:
        raise NotImplementedError()

    def observe(self, action: ActType, reward: float, info: dict) -> None:
        pass

    def reset(self, seed: int) -> None:
        self.rng = np.random.default_rng(seed)


def run_episode(env: gym.Env, agent: Agent, seed: int):

    (rewards, was_best) = ([], [])

    env.reset(seed=seed)
    agent.reset(seed=seed)

    done = False
    while not done:
        arm = agent.get_action()
        (obs, reward, done, info) = env.step(arm)
        agent.observe(arm, reward, info)
        rewards.append(reward)
        was_best.append(1 if arm == info["best_arm"] else 0)

    rewards = np.array(rewards, dtype=float)
    was_best = np.array(was_best, dtype=int)
    return (rewards, was_best)


def run_agent(env: gym.Env, agent: Agent, n_runs=200, base_seed=1):
    all_rewards = []
    all_was_bests = []
    base_rng = np.random.default_rng(base_seed)
    for n in tqdm(range(n_runs)):
        seed = base_rng.integers(low=0, high=10_000, size=1).item()
        (rewards, corrects) = run_episode(env, agent, seed)
        all_rewards.append(rewards)
        all_was_bests.append(corrects)
    return (np.array(all_rewards), np.array(all_was_bests))


class RandomAgent(Agent):
        
    def get_action(self) -> ActType:
        return self.rng.integers(low = 0, high = self.num_arms)

    def __repr__(self):
        # Useful when plotting multiple agents with `plot_rewards`
        return "RandomAgent"


num_arms = 10
stationary = True
env = gym.make("ArmedBanditTestbed-v0", num_arms=num_arms, stationary=stationary)
agent = RandomAgent(num_arms, 0)
all_rewards, all_corrects = run_agent(env, agent)

print(f"Expected correct freq: {1/10}, actual: {all_corrects.mean():.6f}")
assert np.isclose(all_corrects.mean(), 1/10, atol=0.05), "Random agent is not random enough!"

print(f"Expected average reward: 0.0, actual: {all_rewards.mean():.6f}")
assert np.isclose(all_rewards.mean(), 0, atol=0.05), "Random agent should be getting mean arm reward, which is zero."

print("All tests passed!")

100%|██████████| 200/200 [00:01<00:00, 118.17it/s]

Expected correct freq: 0.1, actual: 0.099565
Expected average reward: 0.0, actual: 0.006279
All tests passed!


In [20]:
class RewardAveraging(Agent):
    def __init__(self, num_arms: int, seed: int, epsilon: float, optimism: float):
        self.optimism = optimism
        self.epsilon = epsilon
        super().__init__(num_arms, seed)

    def get_action(self):
        if self.rng.random() < self.epsilon:
            return self.rng.integers(low = 0, high = self.num_arms)
        else:
            return max(self.arm_to_reward, key = self.arm_to_reward.get)

    def observe(self, action, reward, info):
        self.arm_to_num_pulls[action] += 1
        self.arm_to_reward[action] = self.arm_to_reward[action] + (1 / self.arm_to_num_pulls[action]) * (reward - self.arm_to_reward[action])

    def reset(self, seed: int):
        super().reset(seed)
        self.arm_to_reward = dict([(k, self.optimism) for k in range(self.num_arms)])
        self.arm_to_num_pulls = dict([(k, 0) for k in range(self.num_arms)])

    def __repr__(self):
        # For the legend, when plotting
        return f"RewardAveraging(eps={self.epsilon}, optimism={self.optimism})"


num_arms = 10
stationary = True
names = []
all_rewards = []
env = gym.make("ArmedBanditTestbed-v0", num_arms=num_arms, stationary=stationary)

for optimism in [0, 5]:
    agent = RewardAveraging(num_arms, 0, epsilon=0.01, optimism=optimism)
    (rewards, num_correct) = run_agent(env, agent, n_runs=N_RUNS, base_seed=1)
    all_rewards.append(rewards)
    names.append(str(agent))
    print(agent)
    print(f" -> Frequency of correct arm: {num_correct.mean():.4f}")
    print(f" -> Average reward: {rewards.mean():.4f}")

utils.plot_rewards(all_rewards, names, moving_avg_window=15)

100%|██████████| 200/200 [00:01<00:00, 144.79it/s]


RewardAveraging(eps=0.01, optimism=0)
 -> Frequency of correct arm: 0.5196
 -> Average reward: 1.2578


100%|██████████| 200/200 [00:01<00:00, 144.98it/s]

RewardAveraging(eps=0.01, optimism=5)
 -> Frequency of correct arm: 0.7465
 -> Average reward: 1.4833


In [22]:
class CheatyMcCheater(Agent):
    def __init__(self, num_arms: int, seed: int):
        super().__init__(num_arms, seed)
        
        self.best_arm = self.rng.integers(low = 0, high = self.num_arms)

    def get_action(self):
        return self.best_arm

    def observe(self, action: int, reward: float, info: dict):
        self.best_arm = info["best_arm"]

    def __repr__(self):
        return "Cheater"



cheater = CheatyMcCheater(num_arms, 0)
reward_averaging = RewardAveraging(num_arms, 0, epsilon=0.1, optimism=0)
random = RandomAgent(num_arms, 0)

names = []
all_rewards = []

for agent in [cheater, reward_averaging, random]:
    (rewards, num_correct) = run_agent(env, agent, n_runs=N_RUNS, base_seed=1)
    names.append(str(agent))
    all_rewards.append(rewards)

utils.plot_rewards(all_rewards, names, moving_avg_window=15)

assert (all_rewards[0] < all_rewards[1]).mean() < 0.001, "Cheater should be better than reward averaging"
print("Tests passed!")

100%|██████████| 200/200 [00:01<00:00, 122.54it/s]


Tests passed!


In [31]:
class UCBActionSelection(Agent):
    def __init__(self, num_arms: int, seed: int, c: float, eps: float = 1e-6):
        self.epsilon = eps
        self.c = c
        super().__init__(num_arms, seed)

    def get_action(self):
        if self.rng.random() < self.epsilon:
            return self.rng.integers(low = 0, high = self.num_arms)
        else:
            t = sum(self.N)
            return np.argmax(self.Q + self.c * np.sqrt(np.log(t) / (self.N + self.epsilon)))

    def observe(self, action, reward, info):
        self.N[action] += 1
        self.Q[action] += (reward - self.Q[action]) / self.N[action]

    def reset(self, seed: int):
        super().reset(seed)
        self.N = np.zeros(self.num_arms)
        self.Q = np.zeros(self.num_arms, dtype=float)

    def __repr__(self):
        return f"UCB(c={self.c})"



cheater = CheatyMcCheater(num_arms, 0)
reward_averaging = RewardAveraging(num_arms, 0, epsilon=0.1, optimism=0)
reward_averaging_optimism = RewardAveraging(num_arms, 0, epsilon=0.1, optimism=5)
ucb = UCBActionSelection(num_arms, 0, c=2.0)
random = RandomAgent(num_arms, 0)

names = []
all_rewards = []

for agent in [cheater, reward_averaging, reward_averaging_optimism, ucb, random]:
    (rewards, num_correct) = run_agent(env, agent, n_runs=N_RUNS, base_seed=1)
    names.append(str(agent))
    all_rewards.append(rewards)

utils.plot_rewards(all_rewards, names, moving_avg_window=15)

  0%|          | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_2167/4271882850.py:12: RuntimeWarning:

divide by zero encountered in log

/tmp/ipykernel_2167/4271882850.py:12: RuntimeWarning:

invalid value encountered in sqrt

100%|██████████| 200/200 [00:01<00:00, 123.06it/s]


# Tabular RL & Policy Improvement

In [32]:
class Environment:
    def __init__(self, num_states: int, num_actions: int, start=0, terminal=None):
        self.num_states = num_states
        self.num_actions = num_actions
        self.start = start
        self.terminal = np.array([], dtype=int) if terminal is None else terminal
        (self.T, self.R) = self.build()

    def build(self):
        '''
        Constructs the T and R tensors from the dynamics of the environment.
        Outputs:
            T : (num_states, num_actions, num_states) State transition probabilities
            R : (num_states, num_actions, num_states) Reward function
        '''
        num_states = self.num_states
        num_actions = self.num_actions
        T = np.zeros((num_states, num_actions, num_states))
        R = np.zeros((num_states, num_actions, num_states))
        for s in range(num_states):
            for a in range(num_actions):
                (states, rewards, probs) = self.dynamics(s, a)
                (all_s, all_r, all_p) = self.out_pad(states, rewards, probs)
                T[s, a, all_s] = all_p
                R[s, a, all_s] = all_r
        return (T, R)

    def dynamics(self, state: int, action: int) -> Tuple[Arr, Arr, Arr]:
        '''
        Computes the distribution over possible outcomes for a given state
        and action.
        Inputs:
            state : int (index of state)
            action : int (index of action)
        Outputs:
            states  : (m,) all the possible next states
            rewards : (m,) rewards for each next state transition
            probs   : (m,) likelihood of each state-reward pair
        '''
        raise NotImplementedError

    def render(pi: Arr):
        '''
        Takes a policy pi, and draws an image of the behavior of that policy, if applicable.
        Inputs:
            pi : (num_actions,) a policy
        Outputs:
            None
        '''
        raise NotImplementedError

    def out_pad(self, states: Arr, rewards: Arr, probs: Arr):
        '''
        Inputs:
            states  : (m,) all the possible next states
            rewards : (m,) rewards for each next state transition
            probs   : (m,) likelihood of each state-reward pair
        Outputs:
            states  : (num_states,) all the next states
            rewards : (num_states,) rewards for each next state transition
            probs   : (num_states,) likelihood of each state-reward pair (including zero-prob outcomes.)
        '''
        out_s = np.arange(self.num_states)
        out_r = np.zeros(self.num_states)
        out_p = np.zeros(self.num_states)
        for i in range(len(states)):
            idx = states[i]
            out_r[idx] += rewards[i]
            out_p[idx] += probs[i]
        return (out_s, out_r, out_p)

In [33]:
class Toy(Environment):
    def dynamics(self, state: int, action: int):
        (S0, SL, SR) = (0, 1, 2)
        LEFT = 0
        num_states = 3
        num_actions = 2
        assert 0 <= state < self.num_states and 0 <= action < self.num_actions
        if state == S0:
            if action == LEFT:
                (next_state, reward) = (SL, 1)
            else:
                (next_state, reward) = (SR, 0)
        elif state == SL:
            (next_state, reward) = (S0, 0)
        elif state == SR:
            (next_state, reward) = (S0, 2)
        return (np.array([next_state]), np.array([reward]), np.array([1]))

    def __init__(self):
        super().__init__(num_states=3, num_actions=2)

In [34]:
toy = Toy()

actions = ["a_L", "a_R"]
states = ["S_0", "s_L", "S_R"]

imshow(
    toy.T, # dimensions (s, a, s_next)
    title="Transition probabilities T(s_next | s, a) for toy environment", 
    facet_col=-1, facet_labels=[f"s_next = {s}" for s in states], y=states, x=actions,
    labels = {"x": "Action", "y": "State", "color": "Transition<br>Probability"},
)

imshow(
    toy.R, # dimensions (s, a, s_next)
    title="Rewards R(s, a, s_next) for toy environment", 
    facet_col=-1, facet_labels=[f"s_next = {s}" for s in states], y=states, x=actions,
    labels = {"x": "Action", "y": "State", "color": "Reward"},
)

In [35]:
class Norvig(Environment):
    def dynamics(self, state: int, action: int) -> Tuple[Arr, Arr, Arr]:
        def state_index(state):
            assert 0 <= state[0] < self.width and 0 <= state[1] < self.height, print(state)
            pos = state[0] + state[1] * self.width
            assert 0 <= pos < self.num_states, print(state, pos)
            return pos

        pos = self.states[state]
        move = self.actions[action]
        if state in self.terminal or state in self.walls:
            return (np.array([state]), np.array([0]), np.array([1]))
        out_probs = np.zeros(self.num_actions) + 0.1
        out_probs[action] = 0.7
        out_states = np.zeros(self.num_actions, dtype=int) + self.num_actions
        out_rewards = np.zeros(self.num_actions) + self.penalty
        new_states = [pos + x for x in self.actions]
        for (i, s_new) in enumerate(new_states):
            if not (0 <= s_new[0] < self.width and 0 <= s_new[1] < self.height):
                out_states[i] = state
                continue
            new_state = state_index(s_new)
            if new_state in self.walls:
                out_states[i] = state
            else:
                out_states[i] = new_state
            for idx in range(len(self.terminal)):
                if new_state == self.terminal[idx]:
                    out_rewards[i] = self.goal_rewards[idx]
        return (out_states, out_rewards, out_probs)

    def render(self, pi: Arr):
        assert len(pi) == self.num_states
        emoji = ["⬆️", "➡️", "⬇️", "⬅️"]
        grid = [emoji[act] for act in pi]
        grid[3] = "🟩"
        grid[7] = "🟥"
        grid[5] = "⬛"
        print("".join(grid[0:4]) + "\n" + "".join(grid[4:8]) + "\n" + "".join(grid[8:]))

    def __init__(self, penalty=-0.04):
        self.height = 3
        self.width = 4
        self.penalty = penalty
        num_states = self.height * self.width
        num_actions = 4
        self.states = np.array([[x, y] for y in range(self.height) for x in range(self.width)])
        self.actions = np.array([[0, -1], [1, 0], [0, 1], [-1, 0]])
        self.dim = (self.height, self.width)
        terminal = np.array([3, 7], dtype=int)
        self.walls = np.array([5], dtype=int)
        self.goal_rewards = np.array([1.0, -1])
        super().__init__(num_states, num_actions, start=8, terminal=terminal)

In [ ]:
import einops

#tried anf ailed to vectorize this
def policy_eval_numerical(env: Environment, pi: Arr, gamma=0.99, eps=1e-8, max_iterations=10_000) -> Arr:
    '''
    Numerically evaluates the value of a given policy by iterating the Bellman equation
    Inputs:
        env: Environment
        pi : shape (num_states,) - The policy to evaluate
        gamma: float - Discount factor
        eps  : float - Tolerance
        max_iterations: int - Maximum number of iterations to run
    Outputs:
        value : float (num_states,) - The value function for policy pi
    '''
    num_states = env.num_states
    T = env.T
    R = env.R
    value = np.zeros((num_states,))  

    for i in range(max_iterations):    
        new_v = 
            
        delta = np.abs(new_v - value).sum()
        if delta < eps:
            break
        value = np.copy(new_v)
    else:
        print(f"Failed to converge after {max_iterations} steps.")

    return value

    value_fn_old, value_fn_new = np.zeros_like(pi), np.zeros_like(pi)
    
    for n in range(max_iterations):
        transition, reward = env.T, env.R
        
        transition_after_policy = transition[list(range(pi.shape[0])),pi,:]
        reward_after_policy = reward[list(range(pi.shape[0])),pi,:]

        value_fn_new = einops.einsum(transition_after_policy, reward_after_policy + gamma * value_fn_old, "s s_prime, s s_prime -> s")
        value_fn_old = np.copy(value_fn_new)
        
        if np.abs(value_fn_new - value_fn_old).sum() < eps:
            return value_fn_new
    
    return value_fn_new

tests.test_policy_eval(policy_eval_numerical, exact=False)

In [66]:
def policy_eval_numerical(env: Environment, pi: Arr, gamma=0.99, eps=1e-8, max_iterations=10_000) -> Arr:
    '''
    Numerically evaluates the value of a given policy by iterating the Bellman equation
    Inputs:
        env: Environment
        pi : shape (num_states,) - The policy to evaluate
        gamma: float - Discount factor
        eps  : float - Tolerance
        max_iterations: int - Maximum number of iterations to run
    Outputs:
        value : float (num_states,) - The value function for policy pi
    '''
    # SOLUTION
    num_states = env.num_states
    T = env.T
    R = env.R
    value = np.zeros((num_states,))  

    for i in range(max_iterations):    
        new_v = np.zeros_like(value)
        for s in range(num_states):
            new_v[s] = np.dot(T[s, pi[s]], R[s, pi[s]] + gamma * value)
        delta = np.abs(new_v - value).sum()
        if delta < eps:
            break
        value = np.copy(new_v)
    else:
        print(f"Failed to converge after {max_iterations} steps.")

    return value

tests.test_policy_eval(policy_eval_numerical, exact=False)

Converged in 125 steps.
Converged in 123 steps.
Converged in 118 steps.
Converged in 106 steps.
Converged in 128 steps.
All tests in `test_policy_eval` passed!


In [94]:
def policy_eval_exact(env: Environment, pi: Arr, gamma=0.99) -> Arr:
    '''
    Finds the exact solution to the Bellman equation.
    '''
    num_states = env.num_states
    transition, reward = env.T, env.R

    prob_transition_mat = transition[np.arange(num_states),pi,:]
    reward_transition_mat = reward[np.arange(num_states),pi,:]
    
    r = (prob_transition_mat @ reward_transition_mat.T).diagonal()
    v = np.linalg.inv(np.identity(num_states) - gamma * prob_transition_mat) @ r
    
    return v

tests.test_policy_eval(policy_eval_exact, exact=True)

All tests in `test_policy_eval` passed!


In [100]:
def policy_improvement(env: Environment, V: Arr, gamma=0.99) -> Arr:
    '''
    Inputs:
        env: Environment
        V  : (num_states,) value of each state following some policy pi
    Outputs:
        pi_better : vector (num_states,) of actions representing a new policy obtained via policy iteration
    '''
    num_states = env.num_states
    
    T = env.T
    R = env.R

    new_p = np.zeros_like(V, dtype = int)
    
    # V_right_shape = np.zeros((4, num_states))
    
    for s in range(num_states):
        # V_right_shape[:, s] = V
        #the place this is wrong is im confusing s and s_prime
        update = einops.einsum(T[:, :, s], R[:, :, s] + gamma * V[s], "a n_s, a n_s -> a") #but this needs to be s_prime
        
        new_p[s] = np.argmax(update) #this needs to be s
    
    print(new_p)
    return new_p

tests.test_policy_improvement(policy_improvement)

[ 0  1  1  2  4  0 10  0  8 10  9  0]
[ 0  1  1  2  4  0 10  0  8 10  9  0]
[ 0  1  1  2  4  0 10  0  8  9  9 10]
[ 0  1  1  2  4  0 10  0  8  9  9 10]
[0 1 1 2 4 0 0 7 0 0 0 0]
[0 1 1 2 4 0 0 7 0 0 0 0]
[2 3 0 2 1 0 0 0 0 0 0 0]
[2 3 0 2 1 0 0 0 0 0 0 0]
[2 3 0 2 1 0 0 0 0 0 0 0]
[2 3 0 2 1 0 0 0 0 0 0 0]
All tests in `test_policy_improvement` passed!


In [102]:
def policy_improvement(env: Environment, V: Arr, gamma=0.99) -> Arr:
    '''
    Inputs:
        env: Environment
        V  : (num_states,) value of each state following some policy pi
    Outputs:
        pi_better : vector (num_states,) of actions representing a new policy obtained via policy iteration
    '''
    # SOLUTION
    sum_term = sum([
        einops.einsum(env.T, env.R, "s a s_next, s a s_next -> s a"),
        gamma * einops.einsum(env.T, V, "s a s_next, s_next -> s a")
    ])
    
    pi_new = np.argmax(sum_term, axis=1)
    print(pi_new)
    return pi_new
tests.test_policy_improvement(policy_improvement)

[1 1 1 0 0 0 0 0 0 3 0 3]
[1 1 1 0 0 0 0 0 0 3 0 3]
[1 1 1 0 0 0 0 0 0 1 0 3]
[1 1 1 0 0 0 0 0 0 1 0 3]
[1 1 1 0 0 0 0 0 0 3 3 3]
[1 1 1 0 0 0 0 0 0 3 3 3]
[1 1 1 0 0 0 0 0 1 1 0 3]
[1 1 1 0 0 0 0 0 1 1 0 3]
[1 1 1 0 2 0 0 0 1 1 0 3]
[1 1 1 0 2 0 0 0 1 1 0 3]
All tests in `test_policy_improvement` passed!


In [103]:
def find_optimal_policy(env: Environment, gamma=0.99, max_iterations=10_000):
    '''
    Inputs:
        env: environment
    Outputs:
        pi : (num_states,) int, of actions represeting an optimal policy
    '''
    pi = np.zeros(shape=env.num_states, dtype=int)
    for i in range(max_iterations):
        value = policy_eval_exact(env, pi, gamma)
        
        pi = policy_improvement(env, value, gamma)
        
        # if np.abs(value - policy_eval_exact(env, pi, gamma)).sum() < 1e-6:
        #     return pi
        
    return pi


tests.test_find_optimal_policy(find_optimal_policy)

penalty = -0.04
norvig = Norvig(penalty)
pi_opt = find_optimal_policy(norvig, gamma=0.99)
norvig.render(pi_opt)

All tests in `test_find_optimal_policy` passed!
➡️➡️➡️🟩
⬆️⬛⬆️🟥
⬆️⬅️⬆️⬅️
